<a href="https://colab.research.google.com/github/MohammedHamood/20NewsGroup/blob/main/20NewsGroupDT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 20NewsGroup - Logistic Regression

MEMO: Include any decisions about training/validation split, regularization
strategies, any optimization tricks, setting hyper-parameters, etc.

## Data Pre-Processing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin



import time
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

import pipelines_FEngineering as BaseLine
from sklearn.datasets import fetch_20newsgroups
import preprocessingNLP as PNLP

from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA, NMF,TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import Normalizer

# Import Dataset
TNGD_Train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42,remove=['headers', 'footers', 'quotes'])
TNGD_test = fetch_20newsgroups(subset='test',shuffle=True, random_state=42,remove=['headers', 'footers', 'quotes'])

# Preprocessing
print("PREPROCESSING ...")
TNGD_Train.data = PNLP.customNLP(TNGD_Train.data)
TNGD_test.data = PNLP.customNLP(TNGD_test.data)
TNGD_Train.data, TNGD_Train.target = PNLP.removeEmptyInstances(TNGD_Train.data, TNGD_Train.target)



print("PREPROCESSING DONE!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\umroot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\umroot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\umroot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24

PREPROCESSING ...


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b' ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


PREPROCESSING DONE!


In [ ]:
class dcdistanceTransformer(TransformerMixin, BaseEstimator): 
        """
        proposed method modifed from(DCDistance: A Supervised Text Document Feature extraction based on class labels)
        """
        def __init__(self):
            super(dcdistanceTransformer, self).__init__()
            

        def fit(self, X, y, **fit_params):
            self.alpha=20
            self.Classes=np.unique(y, return_index = False)
            self.split=np.linspace(0, X.shape[1],self.alpha,endpoint = False,  dtype=int)
            self.vd=np.zeros((len(self.Classes),X.shape[1]))
            self.vd2=np.zeros((len(self.Classes),X.shape[1]))
            for i in self.Classes:
                ind = np.where(y == i)[0]
                temp=np.zeros((1,X.shape[1]))
                for feat in range(0,len(self.split)):
                    ind1=self.split[feat]              
                    ind2=self.split[feat]+self.split[1]
                    temp=X.tocsr()[ind,ind1:ind2].sum(axis = 0)
                    self.vd2[int(i),ind1:ind2]=temp     
            tfidf_transformer = TfidfTransformer()
            self.vd2 = tfidf_transformer.fit_transform(self.vd2) 
            self.vd2=self.vd2.todense()   
            return self

        def transform(self, X, **fit_params):
            V= np.zeros((X.shape[0],len(self.Classes)))

            
            for i in self.Classes:
                starting=0
                ending=200
                while starting<X.shape[0]-2:
                    if ending>X.shape[0]:
                        ending=X.shape[0]
                    temp=np.linalg.norm(X.tocsr()[starting:ending,:]-self.vd2[int(i),:],axis=1,ord=2)
                    V[starting:ending,int(i)]= temp[:] #('Norm',Normalizer(copy=False)),
                    starting=ending
                    ending=ending+200
            V= sparse.csr_matrix(V)
            X=[]
            return V         

## Cross-Validation Splitting Strategy

Let's first analyse how the cross-validation splitting strategy makes an impact on the training accuracy of the dataset using Logistic Regression classifier with its default parameters. In the following experiment, Logistic Regression will be applied on the dataset for different number of folds (5 to 15). The elapsed time will also be collected for each training.

In [ ]:
# Decision Tree Model
accuracies = []
folds = []
runtimes = []
parameters={}
pip = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                ('tfidf', TfidfTransformer()),
                ('redu',dcdistanceTransformer),('Norm', Normalizer(copy=False)),
                ('clf', DecisionTreeClassifier())])
for i in range(6,13):
  start = time.perf_counter()
  DT_CV = RandomizedSearchCV(pip, parameters, cv=i, n_jobs=-1)
  end = time.perf_counter()
  elapsed_time = end-start
  accuracies.append(DT_CV.best_score_)
  folds.append(i)
  runtimes.append(elapsed_time)

#Display Results
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.plot(folds, accuracies, 'o-')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Fold Size')
ax2.plot(folds, runtimes, 'o-')
ax2.set_ylabel('Training Time')
ax2.set_xlabel('Fold Size')
plt.tight_layout()
plt.show()

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_score_'

The first graph shows that within a range of 5 to 15 folds, a 11-fold cross-validation gives the best accuracy.

## Setting Hyper-Parameters-Pruning

In [ ]:


# Set relevant parameters
parameters = {
    'clf__criterion': ('gini','entropy'),
    'clf__ccp_alpha': (0,.05,.1 ,.5,1,5),

    }

# Create a pipeline
pip = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                ('tfidf', TfidfTransformer()),
                ('redu',dcdistanceTransformer),('Norm', Normalizer(copy=False)),
                ('clf', DecisionTreeClassifier())])

# Initialize RandomizedSearchCV
n_iter_search=8
cv_folds=10
DT_pruning = RandomizedSearchCV(pip, param_distributions=parameters, 
                               n_iter=n_iter_search, cv=cv_folds)

# Execute RandomizedSearchCV and print best results 
DT_pruning.fit(TNGD_Train.data, TNGD_Train.target)
report(DT_pruning.cv_results_)

In [ ]:
## Setting Hyper-Parameters early stoping

In [ ]:

# Set relevant parameters
parameters = {
    'clf__criterion': ('gini','entropy'),
    #'clf__ccp_alpha': (0,.05,.1 ,.5,1,5),
    'clf__max_depth': (None,5,10,20),
    'clf__min_samples_split': (2,8,10,50,100),
    'clf__min_samples_leaf': (1,10,50,100),
    'clf__max_features': (None,'auto', 'sqrt', 'log2'),
#    'clf__max_leaf_nodes': (None,20,30,40,80),
    }

# Create a pipeline
pip = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                ('tfidf', TfidfTransformer()),
                ('redu',dcdistanceTransformer),('Norm', Normalizer(copy=False)),
                ('clf', DecisionTreeClassifier())])

# Initialize RandomizedSearchCV
n_iter_search=10
cv_folds=10
DT_early_stoping = RandomizedSearchCV(pip, param_distributions=parameters, 
                               n_iter=n_iter_search, cv=cv_folds)

# Execute RandomizedSearchCV and print best results 
DT_early_stoping.fit(TNGD_Train.data, TNGD_Train.target)
report(DT_early_stoping.cv_results_)

In the following section, GridSearchCV is used to properly detect the most optimal parameters needed to optimize the validation accuracy of Logistic Regression. 

The previous results help to properly select the hyper-parameters. It seems that the default tolerance of 0.0001 is a good value for stochastic gradient descent. It also seems that a higher validation accuracy is obtained when no penalty is applied on the training. Although this might look good, it could also be a sign of overfitting, which could then generate a poor test accuracy. To avoid this issue, Lasso Regression (L1 penalty) is applied with an appropriate regularization strength. Indeed, this type of regression works the best because it produces sparse weights, which helps with feature selection by reducing the amount of features in the provided design matrix. This eventually reduces the complexity of the model. Furthermore, every test was able to find the minimum error without reaching the maximum iteration provided in the paremeters. Therefore, 300 iterations seems like an appropriate maximum value. Finally, using GridSearchCV helps choosing the appropriate regularization strength for the training.

In [ ]:
# Set relevant parameters
parameters = {
    'clf__C': ( 2.0,5, 10,15)
}

# Create a pipeline
pip = Pipeline([('vect', CountVectorizer(ngram_range=ngram_range, max_features=max_features)),
                ('tfidf', TfidfTransformer()),
                ('Norm', Normalizer(copy=False)),
                ('clf', LogisticRegression(max_iter=300, penalty='l2',tol=.001, solver='saga', n_jobs=-1))])

# Initialize and execute GridSearchCV
gs_clf = GridSearchCV(pip, parameters, cv=cv_folds, n_jobs=-1)
start_time = time.perf_counter()
gs_clf = gs_clf.fit(TNGD_Train.data, TNGD_Train.target)
elapsed_time = time.perf_counter() - start_time

# Print results
print("Validation Accuracy: " + str(gs_clf.best_score_))
print("Optimal Parameters: " + str(gs_clf.best_estimator_))
print("Optimal Parameters: " + str(gs_clf.best_params_))
print("")

train_accuracies = gs_clf.cv_results_['mean_test_score']
train_time = gs_clf.cv_results_['mean_fit_time']
train_params = gs_clf.cv_results_['params']

for i in range(len(train_time)):
  print("Parameter: {0}".format(train_params[i]))
  print("Training Time: %.3f seconds" %train_time[i])
  print("Valdation Accuracy: {0:.3f}".format(train_accuracies[i]))
  print("")

In [ ]:

test_pridect=gs_clf.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

In [ ]:
print(np.mean(test_pridect==TNGD_test.target))